## Final Notebook|| CASCADE CUP '22


### 1. Data Cleaning

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=5)

In [3]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
126,127,0,3,"McMahon, Mr. Martin",male,NaN,0,0,370372,7.7500,NaN,Q
354,355,0,3,"Yousif, Mr. Wazli",male,NaN,0,0,2647,7.2250,NaN,C
590,591,0,3,"Rintamaki, Mr. Matti",male,35.0,0,0,STON/O 2. 3101273,7.1250,NaN,S
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
769,770,0,3,"Gronnestad, Mr. Daniel Danielsen",male,32.0,0,0,8471,8.3625,NaN,S


In [4]:
train_labels=data_train["Survived"]
data_train.loc[data_train['Sex'] =='male','s'] = 1 
data_train.loc[data_train['Sex'] =='female','s'] = 0
data_train=data_train.drop(['PassengerId','Name','Ticket','Cabin','Embarked','Sex','Survived'],axis=1)
data_train.head()

,Pclass,Age,SibSp,Parch,Fare,s
126,3,NaN,0,0,7.7500,1.0
354,3,NaN,0,0,7.2250,1.0
590,3,35.0,0,0,7.1250,1.0
509,3,26.0,0,0,56.4958,1.0
769,3,32.0,0,0,8.3625,1.0


In [5]:
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [6]:
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

###  2. TWEAK THE MODEL

In [7]:
def get_basic_model():

  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(units=10,kernel_initializer='uniform',activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=128,kernel_initializer='uniform',activation='relu'),
    tf.keras.layers.Dropout(rate = 0.2),
    tf.keras.layers.Dense(units=1,kernel_initializer='uniform',activation='sigmoid')
    ])
  #opt = tf.keras.optimizers.SGD(lr=0.001)
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [8]:
ACCURACY_THRESHOLD = 0.9
# Implement callback function to stop training
# when accuracy reaches e.g. ACCURACY_THRESHOLD = 0.95
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True
callbacks = myCallback()


In [10]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=100, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid),callbacks=[callbacks])

Epoch 1/100
 8/23 [=========>....................] - ETA: 0s - loss: 0.6912 - accuracy: 0.6719

2022-02-05 15:06:56.955315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 13ms/step - loss: 0.6874 - accuracy: 0.6390 - val_loss: 0.6721 - val_accuracy: 0.6629
Epoch 2/100
16/23 [===================>..........] - ETA: 0s - loss: 0.6625 - accuracy: 0.6562

2022-02-05 15:06:57.315151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 8ms/step - loss: 0.6595 - accuracy: 0.6489 - val_loss: 0.6199 - val_accuracy: 0.6854
Epoch 3/100
23/23 [==============================] - 0s 9ms/step - loss: 0.5986 - accuracy: 0.6882 - val_loss: 0.5729 - val_accuracy: 0.7191
Epoch 4/100
23/23 [==============================] - 0s 8ms/step - loss: 0.5436 - accuracy: 0.7514 - val_loss: 0.5540 - val_accuracy: 0.7191
Epoch 5/100
23/23 [==============================] - 0s 8ms/step - loss: 0.4917 - accuracy: 0.7781 - val_loss: 0.5299 - val_accuracy: 0.7416
Epoch 6/100
23/23 [==============================] - 0s 8ms/step - loss: 0.4723 - accuracy: 0.7907 - val_loss: 0.5260 - val_accuracy: 0.7753
Epoch 7/100
23/23 [==============================] - 0s 8ms/step - loss: 0.4590 - accuracy: 0.7992 - val_loss: 0.5262 - val_accuracy: 0.7191
Epoch 8/100
23/23 [==============================] - 0s 8ms/step - loss: 0.4527 - accuracy: 0.8090 - val_loss: 0.5166 - val_accuracy: 0.7303
Epoch 9/100
23/23 [======

In [11]:
model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 12ms/step - loss: 0.4254 - accuracy: 0.8000


[0.4253849685192108, 0.800000011920929]

### 3. CSV SUBMISSION

In [ ]:
test=pd.read_csv("data/test.csv")
test.head()

In [ ]:
test=pd.read_csv("data/test.csv")
test_id=test["PassengerId"]
test.loc[test['Sex'] =='male','s'] = 1 
test.loc[test['Sex'] =='female','s'] = 0
test=test.drop(['Name','Ticket','Cabin','Embarked','Sex','PassengerId'],axis=1)
test.head()


In [ ]:
test.to_numpy()
test=np.nan_to_num(test)
ans=model.predict(test)
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"PassengerId",test_id)
data.insert(1,"Survived",ans)


In [ ]:
data.to_csv("data/model1_16.csv",index=False)